In [23]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ruthv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [26]:
# data loading
df= pd.read_csv('twitter_tweets.csv',encoding='ISO-8859-1')

In [27]:
#checking rows and columns
df.shape

(1600000, 6)

In [28]:
# firsgt 5 rows
df.head()

,label,id,timestamp,query,username,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [29]:
#naming the columns

column_names=['target', 'id' ,'date' ,'flag' ,'user' ,'text']

df.columns=column_names

In [30]:
df.shape

(1600000, 6)

In [31]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [32]:
#checking for null values
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [33]:
#checking for distribution in the columns
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [34]:
# converting the target 4 to 1
df.replace({'target':{4:1}},inplace=True)

In [36]:
#checking for distribution in the columns
df['target'].value_counts()
#0-->negative 1-->positive 

target
0    800000
1    800000
Name: count, dtype: int64

In [37]:
#stemming
port_stem=PorterStemmer()

In [ ]:
def stemming(content):

    st_con=re.sub('[^a-zA-Z]',' ',content)
    st_con=st_con.lower()
    st_con=st_con.split()
    st_con=[port_stem.stem(word) for word in st_con if not word in stopwords.words('english')]
    
    return ' '.join(st_con)

In [39]:
df['stemmed_content']=df['text'].apply(stemming)

In [50]:
def part2(l):
    li=" ".join(l)
    return li
df['stemmed_content']=df['stemmed_content'].apply(part2)

In [51]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [52]:
x=df['stemmed_content'].values
y=df['target'].values

In [53]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [54]:
print(y)

[0 0 0 ... 1 1 1]


In [71]:
#splitting training and testing data
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [72]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [73]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [74]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [75]:
#converting textual data to numerical data vector
vectorizer=TfidfVectorizer()

x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [76]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453090 stored elements and shape (1280000, 461490)>
  Coords	Values
  (0, 436715)	0.27259876264838384
  (0, 354545)	0.3588091611460021
  (0, 185194)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235046)	0.41996827700291095
  (0, 443068)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407303)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406401)	0.32105459490875526
  (2, 433562)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443432)	0.3348599670252845
  (2, 266730)	0.24123230668976975
  (2, 409145)	0.15169282335109835
  (2, 178062)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288472)	0.16786949597862733
  (3, 406401)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318305)	0.21254698865277744
  (12

In [77]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461490)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171379)	0.2805816206356074
  (0, 271017)	0.45356623916588285
  (0, 279083)	0.17825180109103442
  (0, 388350)	0.2198507607206174
  (0, 398908)	0.34910438732642673
  (0, 409145)	0.3143047059807971
  (0, 420986)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217563)	0.40288153995289894
  (1, 256778)	0.28751585696559306
  (1, 348137)	0.4739279595416274
  (1, 366205)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183313)	0.5892069252021465
  (2, 256835)	0.2564939661498776
  :	:
  (319994, 443796)	0.2782185641032538


In [78]:
#training the logestic regression model
model=LogisticRegression(max_iter=1000)

In [79]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [80]:
#model evaluation i.e accuracy score on the training data
x_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(y_train,x_train_prediction)

In [81]:
print('accuraacy on train data:',training_data_accuracy)

accuraacy on train data: 0.79871875


In [82]:
#model evaluation i.e accuracy score on the tesing data
x_test_prediction=model.predict(x_test)
testing_data_accuracy=accuracy_score(y_test,x_test_prediction)

In [83]:
print('accuracy on test data',testing_data_accuracy)

accuracy on test data 0.776665625


In [84]:
import pickle

In [85]:
fname='trained_model.sav'
pickle.dump(model,open(fname,'wb'))

In [87]:
#using saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [91]:
x_new=x_test[400]
print(y_test[400])

prediction=loaded_model.predict(x_new)
print(prediction)

if prediction==0:
    print("Negative")
else:
    print("Positive")

1
[1]
Positive
